In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import Python_Chapter_7
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'Python_Chapter_7'

# Data
We import bitcoin data from Yahoo finance from 2021.

In [2]:
btc = yf.Ticker('BTC-USD').history(start='2021-02-03', end='2021-04-01')['Close']

In [3]:
btc_df = pd.DataFrame(btc)
btc_df['Expected 55'] = btc.ewm(span=55, adjust=False).mean()
btc_df['Expected 15'] = btc.ewm(span=15, adjust=False).mean()
btc_df['Target'] = -btc.diff(-1).apply(np.sign).replace(0, -1)
btc_df = btc_df.dropna()
btc= btc_df['Close']

In [4]:
target_positions = btc_df['Target']
data = btc_df[['Close', 'Expected 55', 'Expected 15']]
X_train, X_test, y_train, y_test = train_test_split(data, target_positions, test_size=0.33, shuffle=False)
times = pd.Series(btc.index[1:].union([btc.index[-1] + (btc.index[1] - btc.index[0])]), index=btc.index)

# Purged Training Times
Purges test observations in the training set

In [5]:
Python_Chapter_7.purged_train_times(times, times[X_test.index])

Date
2021-02-02   2021-02-03
2021-02-03   2021-02-04
2021-02-04   2021-02-05
2021-02-05   2021-02-06
2021-02-06   2021-02-07
2021-02-07   2021-02-08
2021-02-08   2021-02-09
2021-02-09   2021-02-10
2021-02-10   2021-02-11
2021-02-11   2021-02-12
2021-02-12   2021-02-13
2021-02-13   2021-02-14
2021-02-14   2021-02-15
2021-02-15   2021-02-16
2021-02-16   2021-02-17
2021-02-17   2021-02-18
2021-02-18   2021-02-19
2021-02-19   2021-02-20
2021-02-20   2021-02-21
2021-02-21   2021-02-22
2021-02-22   2021-02-23
2021-02-23   2021-02-24
2021-02-24   2021-02-25
2021-02-25   2021-02-26
2021-02-26   2021-02-27
2021-02-27   2021-02-28
2021-02-28   2021-03-01
2021-03-01   2021-03-02
2021-03-02   2021-03-03
2021-03-03   2021-03-04
2021-03-04   2021-03-05
2021-03-05   2021-03-06
2021-03-06   2021-03-07
2021-03-07   2021-03-08
2021-03-08   2021-03-09
2021-03-09   2021-03-10
2021-03-10   2021-03-11
Name: Date, dtype: datetime64[ns]

# Embargo Times
Gets embargo time for each bar

In [6]:
Python_Chapter_7.embargo_times(data.index, 0.05)

Date
2021-02-02   2021-02-04
2021-02-03   2021-02-05
2021-02-04   2021-02-06
2021-02-05   2021-02-07
2021-02-06   2021-02-08
2021-02-07   2021-02-09
2021-02-08   2021-02-10
2021-02-09   2021-02-11
2021-02-10   2021-02-12
2021-02-11   2021-02-13
2021-02-12   2021-02-14
2021-02-13   2021-02-15
2021-02-14   2021-02-16
2021-02-15   2021-02-17
2021-02-16   2021-02-18
2021-02-17   2021-02-19
2021-02-18   2021-02-20
2021-02-19   2021-02-21
2021-02-20   2021-02-22
2021-02-21   2021-02-23
2021-02-22   2021-02-24
2021-02-23   2021-02-25
2021-02-24   2021-02-26
2021-02-25   2021-02-27
2021-02-26   2021-02-28
2021-02-27   2021-03-01
2021-02-28   2021-03-02
2021-03-01   2021-03-03
2021-03-02   2021-03-04
2021-03-03   2021-03-05
2021-03-04   2021-03-06
2021-03-05   2021-03-07
2021-03-06   2021-03-08
2021-03-07   2021-03-09
2021-03-08   2021-03-10
2021-03-09   2021-03-11
2021-03-10   2021-03-12
2021-03-11   2021-03-13
2021-03-12   2021-03-14
2021-03-13   2021-03-15
2021-03-14   2021-03-16
2021-03-15 

# Purged KFold and Cross Validation Score
Uses the PurgedKFold struct and functions to achieve CV score

In [7]:
purged_kfold = Python_Chapter_7.PurgedKFold(5, times, 0.05)

In [8]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)

In [9]:
Python_Chapter_7.cross_validation_score(classifier, data=data, labels=target_positions, scoring='neg_log_loss', times=times, cross_validation_generator=purged_kfold, percent_embargo=0.05)

array([-0.68823623, -0.73479345, -0.69952012, -0.73968756, -0.71422815])